In [1]:
import sys
import base64
import requests
import json
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
from tqdm import tqdm
import time

# Mathpix API keys (created in Mathpix website)
app_id = "NEED TO REWRITE"
app_key = "NEED TO REWRITE"

# Define data path
# CSV data for each batch of screenshots are generated in the following cells, 
# and is stored in '~/Documents/Github/Project-S-data-preparation/observations/generated_data'
general_path = '~/Documents/Github/Project-S-data-preparation/observations/'

Berkeley_moon_data_path = general_path + 'Berkeley/Moon/'
Berkeley_sun_a_data_path = general_path + 'Berkeley/Sun A/'
Berkeley_sun_b_data_path = general_path + 'Berkeley/Sun B/'

Shanghai_moon_data_path = general_path + 'Shanghai/Moon/'
Shanghai_sun_a_data_path = general_path + 'Shanghai/Sun A/'
Shanghai_sun_b_data_path = general_path + 'Shanghai/Sun B/'


Sydney_moon_data_path = general_path + 'Sydney/Moon/'
Sydney_sun_a_data_path = general_path + 'Sydney/Sun A/'
Sydney_sun_b_data_path = general_path + 'Sydney/Sun B/'

tmp_save_path = general_path + 'test.png'

# Define functions to recognize numbers in screenshots
def debug(result):
    output = result.replace('=', '-')
    output = output.replace('\\cdot ', '-')
    return output

def mathpix_ocr(file_path):
    for attempt in range(3):
        try:
            image_uri = "data:image/png;base64," + base64.b64encode(open(file_path, "rb").read()).decode()
            r = requests.post("https://api.mathpix.com/v3/text",
            data=json.dumps({'src': image_uri}),
            headers={"app_id": app_id, "app_key": app_key,
                 "Content-type": "application/json"})
            result = json.loads(r.text)['text']
            result = debug(result)
            return result
        except KeyError:
            print('Key Error happened')

def post_process_general(result):
#     output = re.sub("[\{].*?[\}]", "||", result)
    output = np.array(re.findall(r"[-+]?\d*\.?\d+|[-+]?\d+", result))
    output = output.astype(np.float)
    return output

def post_process_day(result):
    output = np.array(re.findall(r'\d+', result))
    output = output.astype(np.float)
    return output

def direction_and_height(load_path):
    lena = mpimg.imread(load_path)
    lenb = lena[298:312,130:376, :] # direction & height #
    plt.imsave(tmp_save_path,lenb)
    output = mathpix_ocr(tmp_save_path)
    output = post_process_general(output)
    return output

def phase(load_path):
    lena = mpimg.imread(load_path)
    lenb = lena[338:353,155:196,:] # phase #
    plt.imsave(tmp_save_path,lenb)
    output = mathpix_ocr(tmp_save_path)
    output = post_process_general(output)
    return output

def day(load_path):
    lena = mpimg.imread(load_path)
    lenb = lena[1414:1435,50:98,:] # day #
    plt.imsave(tmp_save_path,lenb)
    output = mathpix_ocr(tmp_save_path)
    output = post_process_day(output)
    return output

def clock(load_path):
    lena = mpimg.imread(load_path)
    lenb = lena[1414:1435,98:117,:] # clock h #
    plt.imsave(tmp_save_path,lenb)
    h = mathpix_ocr(tmp_save_path)
    h = post_process_general(h)
    
    lenb = lena[1414:1435,122:140,:] # clock m #
    plt.imsave(tmp_save_path,lenb)
    m = mathpix_ocr(tmp_save_path)
    m = post_process_general(m)
    
    lenb = lena[1414:1435,147:182,:] # clock s #
    plt.imsave(tmp_save_path,lenb)
    s = mathpix_ocr(tmp_save_path)
    s = post_process_general(s)
    
    output = np.concatenate((h, m, s))
    return output

def apparent_moon(load_path):
    lena = mpimg.imread(load_path)
    lenb = lena[358:374,155:240, :] # apparent moon #
    plt.imsave(tmp_save_path,lenb)
    output = mathpix_ocr(tmp_save_path)
    output = post_process_general(output)
    return output

def apparent_sun(load_path):
    lena = mpimg.imread(load_path)
    lenb = lena[418:434,155:240, :] # apparent moon #
    plt.imsave(tmp_save_path,lenb)
    output = mathpix_ocr(tmp_save_path)
    output = post_process_general(output)
    return output

def moon_data_generation(path):
    data = None
    error_list = []
    img_list = os.listdir(path)
    img_list.sort()
#     img_list = img_list[596:602] # test #
    for img in tqdm(img_list):
        fig_index = int(img.split('f')[1].split('.')[0])
        try:
            moon_phase = phase(path + img)
            d_and_h = direction_and_height(path + img)
            time_day = day(path + img)
            time_clock = clock(path + img)
            size_moon = apparent_moon(path + img)
            current_result = np.concatenate((np.array([fig_index]), time_day, time_clock, d_and_h, size_moon, moon_phase))
            if data is None:
                data = current_result
            else:
                data = np.vstack((data, current_result))
        except:
            print('Error at ', fig_index)
            error_list.append(fig_index)
            pass
    return data, error_list

def sun_data_generation(path):
    data = None
    error_list = []
    img_list = os.listdir(path)
    img_list.sort()
#     img_list = img_list[2164:2167] # test #
    for img in tqdm(img_list):
        fig_index = int(img.split('f')[1].split('.')[0])
        try:
            d_and_h = direction_and_height(path + img)
            time_day = day(path + img)
            time_clock = clock(path + img)
            size_sun = apparent_sun(path + img)
            current_result = np.concatenate((np.array([fig_index]), time_day, time_clock, d_and_h, size_sun))
            if data is None:
                data = current_result
            else:
                data = np.vstack((data, current_result))
        except:
            print('Error at ', fig_index)
            error_list.append(fig_index)
            pass
    return data, error_list



# Berkeley moon data generation

In [2]:

berkeley_moon_data, berkeley_moon_error_list = moon_data_generation(Berkeley_moon_data_path)


  3%|▎         | 82/3014 [03:04<1:42:52,  2.11s/it]

Error at  81


  6%|▌         | 176/3014 [06:45<1:53:32,  2.40s/it]

Error at  175


  9%|▉         | 286/3014 [11:02<1:49:52,  2.42s/it]

...

 92%|█████████▏| 2764/3014 [1:44:20<07:58,  1.91s/it]

Error at  2763


 96%|█████████▋| 2906/3014 [1:49:08<03:43,  2.07s/it]

Error at  2905


100%|██████████| 3014/3014 [1:53:04<00:00,  2.25s/it]


In [145]:
formt = '%d %d %d %d %d %d %.2f %.0f %d %.2f %.0f %d %.2f %.0f %d %.2f %.3f'
header = 'index year month day hour minute seconad azi_deg azi_min azi_sec pol_deg pol_min pol_sec size_deg size_min size_sec phase'
data_save_path = general_path + 'generated_data/Berkeley/berkeley_moon.csv'
np.savetxt(data_save_path, berkeley_moon_data, fmt = formt, header = header)


# Berkeley sun A data generation

In [168]:

berkeley_sun_a_data, berkeley_sun_a_error_list = sun_data_generation(Berkeley_sun_a_data_path)


  0%|          | 2/3016 [00:03<1:33:40,  1.86s/it]

Error at  1


 22%|██▏       | 663/3016 [20:48<1:17:32,  1.98s/it]

Error at  662


 23%|██▎       | 701/3016 [22:04<1:21:29,  2.11s/it]

...

 89%|████████▉ | 2698/3016 [1:30:26<10:56,  2.06s/it]

Error at  2697


 91%|█████████ | 2746/3016 [1:32:04<08:55,  1.98s/it]

Error at  2745


100%|██████████| 3016/3016 [1:40:57<00:00,  2.01s/it]


In [169]:
formt = '%d %d %d %d %d %d %.2f %.0f %d %.2f %.0f %d %.2f %.0f %d %.2f'
header = 'index year month day hour minute second azi_deg azi_min azi_sec pol_deg pol_min pol_sec size_deg size_min size_sec'
data_save_path = general_path + 'generated_data/Berkeley/berkeley_sun_a.csv'
np.savetxt(data_save_path, berkeley_sun_a_data, fmt = formt, header = header)

# Berkeley sun B data generation

In [170]:

berkeley_sun_b_data, berkeley_sun_b_error_list = sun_data_generation(Berkeley_sun_b_data_path)


 14%|█▍        | 432/3020 [16:15<1:35:43,  2.22s/it]

Error at  431


 18%|█▊        | 532/3020 [19:48<1:19:12,  1.91s/it]

Error at  531


 28%|██▊       | 838/3020 [30:19<1:22:14,  2.26s/it]

...

 93%|█████████▎| 2806/3020 [1:37:47<07:04,  1.99s/it]

Error at  2805


 96%|█████████▌| 2897/3020 [1:40:45<04:00,  1.95s/it]

Error at  2896


100%|██████████| 3020/3020 [1:44:48<00:00,  2.08s/it]


In [171]:
formt = '%d %d %d %d %d %d %.2f %.0f %d %.2f %.0f %d %.2f %.0f %d %.2f'
header = 'index year month day hour minute second azi_deg azi_min azi_sec pol_deg pol_min pol_sec size_deg size_min size_sec'
data_save_path = general_path + 'generated_data/Berkeley/berkeley_sun_b.csv'
np.savetxt(data_save_path, berkeley_sun_b_data, fmt = formt, header = header)

# Shanghai moon data generation

In [172]:

shanghai_moon_data, shanghai_moon_error_list = moon_data_generation(Shanghai_moon_data_path)


  1%|          | 36/3015 [01:22<1:52:15,  2.26s/it]

Error at  35


  3%|▎         | 88/3015 [03:20<1:47:42,  2.21s/it]

Error at  87


 39%|███▉      | 1185/3015 [43:32<1:04:24,  2.11s/it]

...

 89%|████████▉ | 2681/3015 [1:37:26<11:47,  2.12s/it]

Error at  2680


 94%|█████████▍| 2832/3015 [1:42:47<06:18,  2.07s/it]

Error at  2831


100%|██████████| 3015/3015 [1:49:28<00:00,  2.18s/it]


In [173]:
formt = '%d %d %d %d %d %d %.2f %.0f %d %.2f %.0f %d %.2f %.0f %d %.2f %.3f'
header = 'index year month day hour minute seconad azi_deg azi_min azi_sec pol_deg pol_min pol_sec size_deg size_min size_sec phase'
data_save_path = general_path + 'generated_data/Shanghai/shanghai_moon.csv'
np.savetxt(data_save_path, shanghai_moon_data, fmt = formt, header = header)


# Shanghai sun A data generation

In [174]:

shanghai_sun_a_data, shanghai_sun_a_error_list = sun_data_generation(Shanghai_sun_a_data_path)


  6%|▋         | 191/3016 [05:51<1:18:57,  1.68s/it]

Error at  190


 10%|█         | 312/3016 [09:41<1:28:25,  1.96s/it]

Error at  311


 12%|█▏        | 360/3016 [11:10<1:22:12,  1.86s/it]

...

 98%|█████████▊| 2948/3016 [1:33:23<01:58,  1.74s/it]

Error at  2947


100%|█████████▉| 3007/3016 [1:35:16<00:17,  1.98s/it]

Error at  3006


100%|██████████| 3016/3016 [1:35:35<00:00,  1.90s/it]


In [175]:
formt = '%d %d %d %d %d %d %.2f %.1f %d %.2f %.1f %d %.2f %.1f %d %.2f'
header = 'index year month day hour minute second azi_deg azi_min azi_sec pol_deg pol_min pol_sec size_deg size_min size_sec'
data_save_path = general_path + 'generated_data/Shanghai/shanghai_sun_a.csv'
np.savetxt(data_save_path, shanghai_sun_a_data, fmt = formt, header = header)

# Shanghai sun B data generation

In [176]:

shanghai_sun_b_data, shanghai_sun_b_error_list = sun_data_generation(Shanghai_sun_b_data_path)


  0%|          | 14/3016 [00:30<2:36:42,  3.13s/it]

Key Error happened
Key Error happened


  0%|          | 15/3016 [00:32<2:08:21,  2.57s/it]

Key Error happened
Error at  14


  2%|▏         | 59/3016 [01:53<1:31:48,  1.86s/it]

...

 90%|████████▉ | 2705/3016 [1:35:48<09:33,  1.84s/it]

Error at  2704


 98%|█████████▊| 2948/3016 [1:43:34<02:10,  1.92s/it]

Error at  2947


100%|██████████| 3016/3016 [1:45:35<00:00,  2.10s/it]


In [177]:
formt = '%d %d %d %d %d %d %.2f %.0f %d %.2f %.0f %d %.2f %.0f %d %.2f'
header = 'index year month day hour minute second azi_deg azi_min azi_sec pol_deg pol_min pol_sec size_deg size_min size_sec'
data_save_path = general_path + 'generated_data/Shanghai/shanghai_sun_b.csv'
np.savetxt(data_save_path, shanghai_sun_b_data, fmt = formt, header = header)

# Sydney moon data generation

In [178]:

sydney_moon_data, sydney_moon_error_list = moon_data_generation(Sydney_moon_data_path)


 13%|█▎        | 388/3010 [13:15<1:28:53,  2.03s/it]

Error at  387


 17%|█▋        | 521/3010 [17:50<1:25:44,  2.07s/it]

Error at  520


 18%|█▊        | 529/3010 [18:06<1:18:57,  1.91s/it]

...

 96%|█████████▌| 2895/3010 [1:42:23<04:02,  2.11s/it]

Error at  2894


 99%|█████████▉| 2975/3010 [1:45:17<01:18,  2.25s/it]

Error at  2974


100%|██████████| 3010/3010 [1:46:30<00:00,  2.12s/it]


In [179]:
formt = '%d %d %d %d %d %d %.2f %.0f %d %.2f %.0f %d %.2f %.0f %d %.2f %.3f'
header = 'index year month day hour minute seconad azi_deg azi_min azi_sec pol_deg pol_min pol_sec size_deg size_min size_sec phase'
data_save_path = general_path + 'generated_data/Sydney/sydney_moon.csv'
np.savetxt(data_save_path, sydney_moon_data, fmt = formt, header = header)


# Sydney sun A data generation

In [180]:

sydney_sun_a_data, sydney_sun_a_error_list = sun_data_generation(Sydney_sun_a_data_path)


  2%|▏         | 46/3016 [01:24<1:29:49,  1.81s/it]

Error at  45


  7%|▋         | 224/3016 [06:59<1:31:07,  1.96s/it]

Error at  223


 13%|█▎        | 388/3016 [12:01<1:18:44,  1.80s/it]

...

 94%|█████████▍| 2830/3016 [1:28:06<05:56,  1.92s/it]

Error at  2829


 98%|█████████▊| 2955/3016 [1:32:15<02:02,  2.01s/it]

Error at  2954


100%|██████████| 3016/3016 [1:34:23<00:00,  1.88s/it]


In [181]:
formt = '%d %d %d %d %d %d %.2f %.0f %d %.2f %.0f %d %.2f %.0f %d %.2f'
header = 'index year month day hour minute second azi_deg azi_min azi_sec pol_deg pol_min pol_sec size_deg size_min size_sec'
data_save_path = general_path + 'generated_data/Sydney/sydney_sun_a.csv'
np.savetxt(data_save_path, sydney_sun_a_data, fmt = formt, header = header)


# Sydney sun B data generation

In [182]:

sydney_sun_b_data, sydney_sun_b_error_list = sun_data_generation(Sydney_sun_b_data_path)


  7%|▋         | 215/3013 [06:52<1:26:42,  1.86s/it]

Error at  214


 13%|█▎        | 402/3013 [12:38<1:20:40,  1.85s/it]

Error at  401


 21%|██        | 624/3013 [19:35<1:14:04,  1.86s/it]

...

 95%|█████████▌| 2870/3013 [1:31:41<04:17,  1.80s/it]

Error at  2869


 99%|█████████▊| 2970/3013 [1:34:48<01:19,  1.84s/it]

Error at  2969


100%|██████████| 3013/3013 [1:36:06<00:00,  1.91s/it]


In [183]:
formt = '%d %d %d %d %d %d %.2f %.0f %d %.2f %.0f %d %.2f %.0f %d %.2f'
header = 'index year month day hour minute second azi_deg azi_min azi_sec pol_deg pol_min pol_sec size_deg size_min size_sec'
data_save_path = general_path + 'generated_data/Sydney/sydney_sun_b.csv'
np.savetxt(data_save_path, sydney_sun_b_data, fmt = formt, header = header)
